In [1]:
using Oceananigans
using BenchmarkTools

In [1]:
using Oceananigans
using Oceananigans.Units: minutes, hour, hours, day
using CairoMakie
using Printf

# Set up the grid
grid = RectilinearGrid(CPU(), size=(64, 64), extent=(64, 64), halo=(3, 3), topology=(Periodic, Flat, Bounded))


# Boundary conditions
buoyancy_flux(x, t, params) = params.initial_buoyancy_flux * exp(-t^4 / (24 * params.shut_off_time^4))
buoyancy_flux_parameters = (initial_buoyancy_flux = 1e-8, shut_off_time = 2hours)
buoyancy_flux_bc = FluxBoundaryCondition(buoyancy_flux, parameters = buoyancy_flux_parameters)

# Define boundary conditions
N² = 1e-4 # s⁻²
buoyancy_gradient_bc = GradientBoundaryCondition(N²)
buoyancy_bcs = FieldBoundaryConditions(top = buoyancy_flux_bc, bottom = buoyancy_gradient_bc)

# Phytoplankton dynamics
growing_and_grazing(x, z, t, P, params) = (params.μ₀ * exp(z / params.λ) - params.m) * P
plankton_dynamics_parameters = (μ₀ = 1/day, λ = 5, m = 0.1/day)
plankton_dynamics = Forcing(growing_and_grazing, field_dependencies = :P, parameters = plankton_dynamics_parameters)

# Set up the model
model = NonhydrostaticModel(
    grid = grid,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)

# Initial conditions
mixed_layer_depth = 32 # m
stratification(z) = z < -mixed_layer_depth ? N² * z : -N² * mixed_layer_depth
noise(z) = 1e-4 * N² * grid.Lz * randn() * exp(z / 4)
initial_buoyancy(x, z) = stratification(z) + noise(z)
set!(model, b = initial_buoyancy, P = 1)

# Set up the simulation
simulation = Simulation(model, Δt = 2minutes, stop_time = 24hours)

# Adaptive time stepping
conjure_time_step_wizard!(simulation, cfl = 1.0, max_Δt = 2minutes)


# Run the simulation


In [2]:

run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (3.362 seconds)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (6.310 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 25.211 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [12]:
grid_gpu1 = RectilinearGrid(GPU(), size=(16, 16), extent=(16, 16), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu1 = NonhydrostaticModel(
    grid = grid_gpu1,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu1 = Simulation(model_gpu1, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu1)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (1.211 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (5.066 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 7.599 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [13]:
grid_gpu2 = RectilinearGrid(GPU(), size=(32, 32), extent=(32, 32), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu2 = NonhydrostaticModel(
    grid = grid_gpu2,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu2 = Simulation(model_gpu2, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu2)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (1.016 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (5.070 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 8.001 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [14]:
grid_gpu3 = RectilinearGrid(GPU(), size=(64, 64), extent=(64, 64), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu3 = NonhydrostaticModel(
    grid = grid_gpu3,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu3 = Simulation(model_gpu3, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu3)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (1.098 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (5.019 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 8.226 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [15]:
grid_gpu4 = RectilinearGrid(GPU(), size=(128, 128), extent=(128, 128), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu4 = NonhydrostaticModel(
    grid = grid_gpu4,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu4 = Simulation(model_gpu4, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu4)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (1.794 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (4.896 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 7.663 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [16]:
grid_gpu5 = RectilinearGrid(GPU(), size=(256, 256), extent=(256, 256), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu5 = NonhydrostaticModel(
    grid = grid_gpu5,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu5 = Simulation(model_gpu5, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu5)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (4.922 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (4.610 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 8.600 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [17]:
grid_gpu6 = RectilinearGrid(GPU(), size=(512, 512), extent=(512, 512), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu6 = NonhydrostaticModel(
    grid = grid_gpu6,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu6 = Simulation(model_gpu6, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu6)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (11.405 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (4.757 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 12.623 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [18]:
grid_gpu7 = RectilinearGrid(GPU(), size=(1024, 1024), extent=(1024, 1024), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu7 = NonhydrostaticModel(
    grid = grid_gpu7,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu7 = Simulation(model_gpu7, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu7)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (19.833 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (4.458 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 30.980 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [20]:
grid_gpu9 = RectilinearGrid(GPU(), size=(2048, 2048), extent=(2048, 2048), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu9 = NonhydrostaticModel(
    grid = grid_gpu9,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu9 = Simulation(model_gpu9, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu9)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (71.825 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (228.210 ms).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 1.881 minutes.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [22]:
grid_gpu10 = RectilinearGrid(GPU(), size=(4096, 4096), extent=(4096, 4096), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_gpu10 = NonhydrostaticModel(
    grid = grid_gpu10,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_gpu10 = Simulation(model_gpu10, Δt = 2minutes, stop_time = 24hours)
run!(simulation_gpu10)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (166.034 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (5.099 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 7.724 minutes.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [23]:
grid_cpu1 = RectilinearGrid(CPU(), size=(16, 16), extent=(16, 16), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_cpu1 = NonhydrostaticModel(
    grid = grid_cpu1,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_cpu1 = Simulation(model_cpu1, Δt = 2minutes, stop_time = 24hours)
run!(simulation_cpu1)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (95.893 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (8.406 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 10.668 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [24]:
grid_cpu2 = RectilinearGrid(CPU(), size=(32, 32), extent=(32, 32), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_cpu2 = NonhydrostaticModel(
    grid = grid_cpu2,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_cpu2 = Simulation(model_cpu2, Δt = 2minutes, stop_time = 24hours)
run!(simulation_cpu2)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (124.825 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (8.659 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 11.550 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [25]:
grid_cpu3 = RectilinearGrid(CPU(), size=(64, 64), extent=(64, 64), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_cpu3 = NonhydrostaticModel(
    grid = grid_cpu3,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_cpu3 = Simulation(model_cpu3, Δt = 2minutes, stop_time = 24hours)
run!(simulation_cpu3)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (79.410 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (4.659 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 10.464 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [26]:
grid_cpu4 = RectilinearGrid(CPU(), size=(128, 128), extent=(128, 128), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_cpu4 = NonhydrostaticModel(
    grid = grid_cpu4,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_cpu4 = Simulation(model_cpu4, Δt = 2minutes, stop_time = 24hours)
run!(simulation_cpu4)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (119.257 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (5.575 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 25.143 seconds.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [27]:
grid_cpu5 = RectilinearGrid(CPU(), size=(256, 256), extent=(256, 256), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_cpu5 = NonhydrostaticModel(
    grid = grid_cpu5,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_cpu5 = Simulation(model_cpu5, Δt = 2minutes, stop_time = 24hours)
run!(simulation_cpu5)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (212.983 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (8.641 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 1.805 minutes.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [28]:
grid_cpu6 = RectilinearGrid(CPU(), size=(512, 512), extent=(512, 512), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_cpu6 = NonhydrostaticModel(
    grid = grid_cpu6,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_cpu6 = Simulation(model_cpu6, Δt = 2minutes, stop_time = 24hours)
run!(simulation_cpu6)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (211.162 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (5.382 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 6.311 minutes.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [29]:
grid_cpu7 = RectilinearGrid(CPU(), size=(1024, 1024), extent=(1024, 1024), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_cpu7 = NonhydrostaticModel(
    grid = grid_cpu7,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_cpu7 = Simulation(model_cpu7, Δt = 2minutes, stop_time = 24hours)
run!(simulation_cpu7)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info:     ... simulation initialization complete (690.176 ms)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (7.909 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:126
┌ Info: Simulation is stopping after running for 26.393 minutes.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:202
┌ Info: Simulation time 1 day equals or exceeds stop time 1 day.
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\simulation.jl:203


In [ ]:
grid_cpu8 = RectilinearGrid(CPU(), size=(2048, 2048), extent=(2048, 2048), halo=(3, 3), topology=(Periodic, Flat, Bounded))

model_cpu8 = NonhydrostaticModel(
    grid = grid_cpu8,
    advection = UpwindBiasedFifthOrder(),
    timestepper = :RungeKutta3,
    closure = ScalarDiffusivity(ν=1e-4, κ=1e-4),
    coriolis = FPlane(f=1e-4),
    tracers = (:b, :P),
    buoyancy = BuoyancyTracer(),
    forcing = (; P = plankton_dynamics),
    boundary_conditions = (; b = buoyancy_bcs)
)
simulation_cpu8 = Simulation(model_cpu8, Δt = 2minutes, stop_time = 24hours)
run!(simulation_cpu8)
